1. Импорт необходимых библиотек

In [1]:
import json
import spacy
from spacy.tokens import DocBin
from spacy.util import filter_spans
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup

2. Загрузка модели и тренировочных данных

In [2]:
# Загрузка модели en_core_web_lg
nlp = spacy.load('en_core_web_lg')

# Загрузка тренировочных данных
with open('datasetFurniture.json', 'r') as f:
    data = json.load(f)

training_data = []

# Формируем тренировочные данные из JSON
for elem in data:
    temp_dict = {}
    temp_dict['text'] = elem['name']
    temp_dict['entities'] = []
    for label in elem['label']:
        start = label['start']
        end = label['end']
        label = label['labels'][0].upper()
        temp_dict['entities'].append((start, end, label))
    training_data.append(temp_dict)

3. Создание пустой модели SpaCy и подготовка документов

In [3]:
# Создание пустой модели Spacy
nlp = spacy.blank('en')

# Подготовка для хранения документов
doc_bin = DocBin()

# Преобразование тренировочных данных в документы Spacy
for training_example in tqdm(training_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode='contract')
        if span is None:
            continue
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

100%|██████████| 136/136 [00:00<00:00, 3159.73it/s]


4. Сохранение в бинарный формат

In [4]:
# Сохранение в бинарный формат
doc_bin.to_disk("train.spacy")

5. Загрузка обученной модели NER

In [5]:
# Загрузка обученной модели NER
nlp_ner = spacy.load("model-best")

6. Функция для парсинга веб-сайта и извлечения сущностей

In [6]:
# Функция для парсинга веб-сайта и извлечения сущностей типа PRODUCT
def parse_website(url):
    # 1. Загрузить HTML-контент с сайта
    response = requests.get(url)

    if response.status_code == 200:
        html_content = response.text
    else:
        print(f"Ошибка при загрузке страницы: {response.status_code}")
        return None

    # 2. Извлечь текст из HTML с помощью BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    texts = soup.stripped_strings  # Извлекаем все текстовые строки
    full_text = " ".join(texts)  # Собираем их в один текст

    # 3. Применить модель NER для извлечения сущностей
    doc = nlp_ner(full_text)

    # 4. Найти все сущности типа PRODUCT и удалить дубликаты
    products = [ent.text for ent in doc.ents if ent.label_ == "PRODUCT"]
    unique_products = list(set(products))  # Удаление дубликатов

    return unique_products

7. Пример использования функции

In [9]:
# Пример использования функции
url = "https://www.factorybuys.com.au/products/euro-top-mattress-king"
products = parse_website(url)
counter = 0
if products:
    print("Найденные сущности типа PRODUCT (без дубликатов):")
    for product in products:
        # Ограничение поля для сокращения вывода в целях читаемости
        if counter < 10:
            print(f"- {product}")
            counter += 1
else:
    print("Сущностей типа PRODUCT не найдено.")

Найденные сущности типа PRODUCT (без дубликатов):
- Wall Clocks Mirrors
- Gas Lift Storage
- Benches Plastic Stools
- Velvet Armchairs
- Bedroom Furniture Bed
- Bedroom Play Mats Change Tables
- Bed Safety
- Double Mattress Toppers
- King Quilt Covers King Sheets King Mattress Protectors King Mattress Toppers
- Pool Accessories Outdoor & Camping Outdoor
